In [1]:
# Import system packages
using Printf
using DelimitedFiles
using Plots
using LaTeXStrings

In [2]:
#            Psurf/kbar         Msurf/Me            Rsurf/Re            Fint/ Wm-2
structure = [36.606020371988    1.89635484051907    1.26977486802543    0.1
             72.7858858655158   1.87155246958974    1.19269414869405    0.1
             203.510277091369   1.82923648889318    1.03435826758376    0.1    ]
nsamps = size(structure)[1]

3

In [3]:
# Import AGNI
ROOT_DIR = abspath(joinpath(pwd(),"../"))
using AGNI
import AGNI.atmosphere as atmosphere
import AGNI.solver as nl
import AGNI.setpt as setpt
import AGNI.dump as dump
import AGNI.plotting as plotting

# Disable logging from AGNI module
AGNI.setup_logging("",1)

In [4]:
R_earth = 6.371e6
M_earth = 5.972e24

5.972e24

In [5]:
# Configuration options
instellation    = 6501.0
mass            = 1.873*M_earth
radius          = 1.265*R_earth
gravity         = 6.67e-11 * mass / radius^2
p_surf          = 5.57e4

s0_fact         = 0.25
zenith_degrees  = 54.74
albedo_b        = 0.0
nlev_centre     = 58
p_top           = 1e-5      # bar
mole_fractions  = Dict([
                        ("H2S", 0.23988329190194904),
                        ("SO2", 0.0044668359215096305),
                        ("N2" , 5.623413251903491e-6),
                        ("H2" , 0.7093907341517219)
                    ])
tmp_surf        = 3100.0
thermo          = true

albedo_data   = joinpath(ROOT_DIR,"res/surface_albedos/lunar_marebasalt.dat")
spectral_file = joinpath(ROOT_DIR,"res/spectral_files/Honeyside/256/Honeyside.sf")
star_file     = joinpath(ROOT_DIR,"res/stellar_spectra/l-98-59.txt")
output_dir    = joinpath(ROOT_DIR,"out/")

"/home/n/nichollsh/AGNI/out/"

In [6]:
# Create output directory
rm(output_dir,force=true,recursive=true)
mkdir(output_dir);

In [7]:
# Setup atmosphere
atmos = atmosphere.Atmos_t()
atmosphere.setup!(atmos, ROOT_DIR, output_dir,
                        spectral_file,
                        instellation, s0_fact, albedo_b, zenith_degrees,
                        tmp_surf,
                        gravity, radius,
                        nlev_centre, p_surf, p_top,
                        mole_fractions, "",

                        flux_int = 0.0,
                        flag_gcontinuum=true,
                        flag_rayleigh=true,
                        thermo_functions=thermo,
                        surface_material=albedo_data
                    )
atmosphere.allocate!(atmos, star_file)

[ INFO  ] Composition set by dict 
[ INFO  ] Found FastChem executable 
[ INFO  ] Inserting stellar spectrum and Rayleigh coefficients 
[ INFO  ] Allocated atmosphere with composition: 
[ INFO  ]       1 H2      7.44e-01  
[ INFO  ]       2 H2S     2.52e-01  
[ INFO  ]       3 SO2     4.68e-03  
[ INFO  ]       4 N2      5.90e-06  


true

In [8]:
setpt.loglinear!(atmos, 700.0)

In [9]:
solver_success = nl.solve_energy!(atmos,
                                    sol_type=1,
                                    sens_heat=true,
                                    latent=true,
                                    method=1,
                                    dx_max=400.0,
                                    ls_method=0,
                                    save_frames=false, modplot=1,
                                    conv_atol=0.5,
                                    modprint=1
                                    )
println("Solver success? $solver_success")
atm_base = deepcopy(atmos);

[ INFO  ]     sol_type = 1 
[ INFO  ]     tmp_surf = 3100.00 K 
[ INFO  ]     step  resid_med    cost     flux_OLR    max(x)    max(|dx|)   flags 
[ INFO  ]        1  -1.06e+03  1.327e+04  7.358e+04  3.100e+03  4.000e+02  C2-Nr 
[ INFO  ]        2  -3.39e+02  4.231e+03  2.463e+04  3.100e+03  3.747e+02  C2-Nr 
[ INFO  ]        3  -8.37e+01  7.106e+05  8.437e+03  3.100e+03  3.604e+02  C2-Nr 
[ INFO  ]        4  -8.41e+00  4.994e+06  3.306e+03  3.100e+03  3.169e+02  C2-Nr 
[ INFO  ]        5  -1.48e-01  1.741e+07  1.732e+03  3.100e+03  2.445e+02  C2-Nr 
[ INFO  ]        6  -1.84e-06  1.514e+07  1.403e+03  3.100e+03  1.003e+02  C2-Nr 
[ INFO  ]        7  +1.01e-07  5.834e+06  1.347e+03  3.100e+03  3.808e+01  C2-Nr 
[ INFO  ]        8  +2.08e-07  1.093e+06  1.339e+03  3.100e+03  1.897e+01  C2-Nr 


InterruptException: InterruptException:

In [10]:
atm_arr = atmosphere.Atmos_t[]

@printf("Running model for %d samples... \n", nsamps)

# atmos = deepcopy(atm_base)

setpt.isothermal!(atmos, 1000.0)
for i in 1:nsamps
    @printf("    sample %d/%d \n",i,nsamps)

    # update planet structure
    row = structure[i,:]
    Ps = row[1] * 1e3 * 1e5 # convert kbar -> bar -> Pa
    Ms = row[2] * M_earth
    Rs = row[3] * R_earth
    Fi = row[4]

    atmos.rp = Rs
    atmos.p_boa = Ps
    atmos.grav_surf = 6.67e-11 * Ms / Rs^2
    atmos.flux_int = Fi
    @printf("    Ps=%.2e Pa  ,  Ms=%.2e kg  ,  Rs=%.2e m  ,  Fi=%.3f Wm-2\n",Ps,Ms,Rs,Fi)

    atmosphere.generate_pgrid!(atmos)

    # Run model
    solver_success = nl.solve_energy!(atmos,
                                    sol_type=3,
                                    method=1,
                                    dx_max=300.0,
                                    ls_method=0,
                                    save_frames=false, modplot=1,
                                    modprint=1, perturb_all=false,
                                    conv_atol=1.0e-2
                                    )

    # Store result
    push!(atm_arr, deepcopy(atmos))
    @printf("--------------------------------- \n")
end
println("Done!")

Running model for 3 samples... 
    sample 1/3 
    Ps=3.66e+09 Pa  ,  Ms=1.13e+25 kg  ,  Rs=8.09e+06 m  ,  Fi=0.100 Wm-2
[ INFO  ]     sol_type = 3 
[ INFO  ]     flux_int = 0.10 W m-2 
[ INFO  ]     step  resid_med    cost     flux_OLR    max(x)    max(|dx|)   flags 
[ INFO  ]        1  -1.98e-02  4.928e+05  4.915e+04  1.301e+03  3.000e+02  C2-Nr 
[ INFO  ]        2  -3.40e+01  5.544e+07  2.124e+04  1.381e+03  3.000e+02  C2-Nr 
[ INFO  ]        3  -2.83e+00  8.061e+06  7.019e+03  1.398e+03  2.149e+02  C2-Nr 
[ INFO  ]        4  -1.33e-02  1.560e+06  2.739e+03  1.386e+03  1.456e+02  C2-Nr 
[ INFO  ]        5  -2.46e-03  2.894e+05  1.385e+03  1.383e+03  1.134e+02  C2-Nr 
[ INFO  ]        6  -1.54e-03  5.131e+04  1.028e+03  1.383e+03  9.094e+01  C2-Nr 
[ INFO  ]        7  -1.33e-05  6.438e+03  9.328e+02  1.383e+03  5.448e+01  C2-Nr 
[ INFO  ]        8  -1.64e-06  2.174e+03  9.113e+02  1.383e+03  2.573e+01  C2-Nr 
[ INFO  ]        9  -1.24e-07  1.065e+03  9.099e+02  1.383e+03  1.317e+01 

InterruptException: InterruptException:

In [11]:
for (i, atm) in enumerate(atm_arr)
    @printf("Writing index %d \n",i)
    dump.write_ncdf(atm, joinpath(atm.OUT_DIR, "$i.nc"))
end
@printf("Done! \n")

Done! 


In [12]:
fs=12
lw=1.8
al=0.8

0.8

In [13]:
# find highest psurf
psurf_max::Float64 = 1e3
psurf_idx::Int = 1
for i in 1:nsamps
    this_atm = atm_arr[i]
    if atm_arr[i].pl[end] > psurf_max
        psurf_max = atm_arr[i].pl[end]
        psurf_idx = i
    end
end

# y ticks
arr_P = atm_arr[psurf_idx].pl .* 1.0e-5 # Convert Pa to bar
ylims  = (arr_P[1]/2, arr_P[end]*2)
yticks = 10.0 .^ round.(Int,range( log10(ylims[1]), stop=log10(ylims[2]), step=1))

plt = plot(framestyle=:box, size=(750,500), dpi=300,
                leg=:outertopright, legcolumn=-1,
                tickfontsize=fs, guidefontsize=fs, legendfontsize=fs,
                legendtitlefontsize=fs,
                leg_title="Index",
                left_margin=4Plots.mm, bottom_margin=4Plots.mm)


# plot the profiles
for i in 1:nsamps
    this_atm = atm_arr[i]
    plot!(plt, this_atm.tmpl, this_atm.pl* 1.0e-5, linewidth=lw, label=@sprintf("%d",i), linealpha=al)
end

# decorate
xlabel!(plt, "Temperature[K]")
xaxis!(plt, minorgrid=true)
ylabel!(plt, "Pressure [bar]")
yflip!(plt)
yaxis!(plt, yscale=:log10, ylims=ylims, yticks=yticks)
display(plt)
savefig(plt,joinpath(output_dir,"profiles.pdf"))

BoundsError: BoundsError: attempt to access 0-element Vector{AGNI.atmosphere.Atmos_t} at index [1]

In [14]:
arr_P = atmos.pl .* 1.0e-5 # Convert Pa to bar
ylims  = (arr_P[1], arr_P[end])
yticks = 10.0 .^ round.(Int,range( log10(ylims[1]), stop=log10(ylims[2]), step=1))

plt = plot(framestyle=:box, size=(750,500), dpi=300,
                leg=:outertopright, legcolumn=-1,
                tickfontsize=fs, guidefontsize=fs, legendfontsize=fs,
                legendtitlefontsize=fs,
                leg_title="Index",
                left_margin=4Plots.mm, bottom_margin=4Plots.mm)


for i in 1:nsamps
    this_atm = atm_arr[i]
    plot!(plt, this_atm.flux_cdry[1:end-2], this_atm.pl[1:end-2]* 1.0e-5,  linewidth=lw, label=@sprintf("%d",i), linealpha=al)
end

xlabel!(plt, "Convective flux [W m-2]")
xaxis!(plt, minorgrid=true)
ylabel!(plt, "Pressure [bar]")
yflip!(plt)
yaxis!(plt, yscale=:log10, ylims=ylims, yticks=yticks)
display(plt)
savefig(plt,joinpath(output_dir,"convection.pdf"))

BoundsError: BoundsError: attempt to access 0-element Vector{AGNI.atmosphere.Atmos_t} at index [1]

In [15]:
arr_P = atmos.pl .* 1.0e-5 # Convert Pa to bar
ylims  = (arr_P[1], arr_P[end])
yticks = 10.0 .^ round.(Int,range( log10(ylims[1]), stop=log10(ylims[2]), step=1))

plt = plot(framestyle=:box, size=(750,500), dpi=300,
                leg=:outertopright, legcolumn=-1,
                tickfontsize=fs, guidefontsize=fs, legendfontsize=fs,
                legendtitlefontsize=fs,
                leg_title="Index",
                left_margin=4Plots.mm, bottom_margin=4Plots.mm)


for i in 1:nsamps
    this_atm = atm_arr[i]
    plot!(plt, this_atm.flux_d_sw, this_atm.pl*1.0e-5,  linewidth=lw, label=@sprintf("%d",i), linealpha=al)
end

xlabel!(plt, "Stellar flux [W m-2]")
xaxis!(plt, minorgrid=true)
ylabel!(plt, "Pressure [bar]")
yflip!(plt)
yaxis!(plt, yscale=:log10, ylims=ylims, yticks=yticks)
display(plt)
savefig(plt,joinpath(output_dir,"fl_D_SW.pdf"))

BoundsError: BoundsError: attempt to access 0-element Vector{AGNI.atmosphere.Atmos_t} at index [1]

In [16]:
plt = plot(framestyle=:box, size=(900,400), dpi=300,
                leg=:outertopright, legcolumn=-1,
                tickfontsize=fs, guidefontsize=fs, legendfontsize=fs,
                legendtitlefontsize=fs,
                leg_title="Index",
                left_margin=4Plots.mm, bottom_margin=4Plots.mm)

for i in 1:nsamps
    this_atm = atm_arr[i]


    # band widths
    wd = atmos.bands_wid * 1e9 # convert to nm

    # band centres
    xe = atmos.bands_cen * 1e9 # convert to nm

    # TOA upward spectral flux [erg s-1 cm-2 nm-1]
    yt = (this_atm.band_u_sw[1, :] .+  this_atm.band_u_lw[1, :]) .* 1000.0 ./ wd

    plot!(plt, xe, yt,  linewidth=lw, label=@sprintf("%d",i), linealpha=al)
end

xlabel!(plt, "Wavelength [nm]")
xaxis!(plt, xscale=:lo010, minorgrid=true, xlims=(300.0, 6.0e3))

ylabel!(plt, "Flux density [W m-2 nm-1]")
yaxis!(plt, yscale=:log10, ylims=(0.1, Inf))

display(plt)
savefig(plt,joinpath(output_dir,"emission.pdf"))

BoundsError: BoundsError: attempt to access 0-element Vector{AGNI.atmosphere.Atmos_t} at index [1]